In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from datetime import datetime

from utility import *
from model_utility import *

In [2]:
LOOKBACK = 61
TRAIN_SPLIT_RATIO = 0.9
columns = [OPEN_INDEX, HIGH_INDEX, LOW_INDEX, CLOSE_INDEX, VOLUME_INDEX]

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
todays_date = datetime.now().strftime('%Y-%m-%d')
download_monthly_klines('spot', ['BTCUSDT'], ['15m'], '2019-01-01', todays_date, 'train')

Found 1 symbols
[1/1] - start download monthly BTCUSDT klines 

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-01.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-02.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-03.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-04.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-05.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-06.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-07.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-08.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-09.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-10.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-11.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2019-12.zip

file already exists! /content/train/BTCUSDT/BTCUSDT-15m-2020-01.zip

file already exists! /content/train/BTC

In [5]:
dataframe = get_data('train', 'BTCUSDT')

In [6]:
dataframe = dataframe[dataframe[IGNORE_INDEX] == 0]
dataframe = dataframe[columns]

scalers = {}
for col in columns:
    scaler = MinMaxScaler().fit(dataframe[[col]])
    dataframe[col] = scaler.transform(dataframe[[col]])
    scalers[col] = scaler

In [7]:
x_train, y_train, x_test, y_test = split_data(dataframe, LOOKBACK, TRAIN_SPLIT_RATIO)

print('x_train.shape = {}'.format(x_train.shape))
print('y_train.shape = {}'.format(y_train.shape))
print('x_test.shape = {}'.format(x_test.shape))
print('y_test.shape = {}'.format(y_test.shape))

x_train.shape = (120668, 60, 5)
y_train.shape = (120668, 1)
x_test.shape = (13407, 60, 5)
y_test.shape = (13407, 1)


In [8]:
x_train = torch.tensor(x_train, dtype=torch.float)
x_test = torch.tensor(x_test, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.float)

In [9]:
input_dim = x_train.shape[1]
output_dim = 1

batch_size = 1000

In [10]:
train_dataset = CryptocurrencyDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [11]:
test_dataset = CryptocurrencyDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
class CNN_LSTM(nn.Module):
    def __init__(self, input_shape, output_shape):
        super(CNN_LSTM, self).__init__()
        
        self.conv1 = nn.Conv1d(5, 5, (5, 3))
        self.lstm1 = nn.LSTM(58, 60)
        self.fc1 = nn.Linear(60, 1)
        self.fc2 = nn.Linear(5, 1)
        
    def observe_outputs(self, x):
        x = self.conv1(x.T)
        print('After conv1: {}'.format(x.size()))
        
        x, _ = self.lstm1(x)
        print('After lstm1: {}'.format(x.size()))
        
        x = self.fc1(x)
        print('After fc1: {}'.format(x.size()))

        x = self.fc2(x.T)
        print('After fc1: {}'.format(x.size()))
        
    def forward(self, x):
        T_x, n_x, m = x.shape

        output = torch.zeros((T_x, 1), device=device)
        for t in range(T_x):
            out = self.conv1(x[t].T)
            out, _ = self.lstm1(out)
            out = self.fc1(out)
            out = self.fc2(out.T)
            output[t, :] = out

        return output      

In [13]:
def test_forward():
    cnn_lstm = CNN_LSTM(input_dim, output_dim).to(device)

    x = torch.randn(16, 60, 5).to(device)
    with torch.no_grad():
        cnn_lstm.observe_outputs(x)

test_forward()

After conv1: torch.Size([5, 56])
After lstm1: torch.Size([5, 60])
After fc1: torch.Size([5, 1])
After fc1: torch.Size([1, 1])


In [14]:
num_params = count_parameters(CNN_LSTM(input_dim, output_dim))
print('Total number of parameters: {}'.format(num_params))

Total number of parameters: 28517


In [15]:
cnn_lstm = CNN_LSTM(input_dim, output_dim).to(device)

optimizer = torch.optim.Adam(cnn_lstm.parameters(), lr=0.001)

loss = nn.MSELoss()

print(cnn_lstm)

CNN_LSTM(
  (conv1): Conv1d(5, 5, kernel_size=(5,), stride=(1,))
  (lstm1): LSTM(56, 60)
  (fc1): Linear(in_features=60, out_features=1, bias=True)
  (fc2): Linear(in_features=5, out_features=1, bias=True)
)


In [ ]:
y_train_pred, history = model_train(cnn_lstm, optimizer, loss, 10, train_loader, device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([668])) that is different to the input size (torch.Size([668, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: ignored

In [ ]:
y_test_pred = model_predictions(cnn_lstm, x_test, y_test, device)

In [ ]:
rmse(y_train, y_train_pred, 'Train')
rmse(y_test, y_test_pred, 'Test')